# WiNDC National Model

You will need to point to the correct locations for a few objects, listed below

In [10]:
parameter_path = raw"C:\Users\mphillipson\Documents\Modeling\MPSGE_dev_testing\nationaldata_julia\DAAData.jld2"
set_path = raw"C:\Users\mphillipson\Documents\Modeling\MPSGE_dev_testing\nationaldata_julia\Indices.jld2";

In [11]:
using MPSGE_MP

using CSV, DataFrames, JLD2

using JuMP

In [12]:
P= load(parameter_path)["data"] # load in date from saved Notebook output Dict, named P
S= load(set_path)["data"] # load in date from saved Notebook output Dict, named P


I = [i for i∈S[:i] if i∉[:use,:oth]]
J = [i for i∈S[:j] if i∉[:use,:oth]]
VA = [va for va∈S[:va] if va!=:othtax]
FD = S[:fd]
TS = S[:ts]
YR = S[:yr]
M = S[:m]

a_0 = P[:a_0]
id_0 = P[:id_0]
ys_0 = P[:ys_0]
tm_0 = P[:tm_0]
va_0 = P[:va_0]
md_0 = P[:md_0]
fd_0 = P[:fd_0]
m_0 = P[:m_0]
ty_0 = P[:ty_0]
ms_0 = P[:ms_0]
bopdef_0 = P[:bopdef_0]
x_0 = P[:x_0]
ta_0 = P[:ta_0]
#s_0 = P[:s_0]
fs_0 = P[:fs_0]
y_0 = P[:y_0];



1;

In [13]:
yr = Symbol(2017)

WiNnat = MPSGEModel()

#y_ = [j for j∈J if sum(ys_0[yr,j,i] for i∈I) !=0]
#a_ = [i_ for i_∈I if a_0[yr,i_]!=0]


@parameters(WiNnat, begin
    ta, ta_0[yr,J], (index = [J],)
    ty, ty_0[yr,J], (index = [J],)
    tm, tm_0[yr,J], (index = [J],)
end)

@sectors(WiNnat,begin
    Y, (index = [J], description = "Sectoral Production")
    A, (index = [I], description = "Armington Supply")
    MS, (index = [M],description = "Margin Supply")
end)

@commodities(WiNnat,begin
    PA, (index = [I],  description = "Armington Price")
    PY, (index = [J],  description = "Supply")
    PVA, (index = [VA],description = "Value-added")
    PM, (index = [M],  description = "Foreign Exachange")
    PFX
end)

@consumer(WiNnat, RA, description = "Representative Agent")

for j∈J
    @production(WiNnat, Y[j], [t=0, s = 0, va => s = 1], begin
        [@Output(PY[i],ys_0[yr,j,i], t, taxes = [Tax(RA,ty[j])]) for i∈I]... 
        [@Input(PA[i], id_0[yr,i,j], s) for i∈I]...
        [@Input(PVA[va], va_0[yr,va,j], va) for va∈VA]...
    end)
end



for m∈M
    @production(WiNnat, MS[m], [t = 0, s = 0], begin
        [@Output(PM[m], sum(ms_0[yr,i,m] for i∈I), t)]...
        [@Input(PY[i], ms_0[yr,i,m], s) for i∈I]...
    end)
end

for i∈I
    @production(WiNnat, A[i], [t = 2, s = 0, dm => s = 2], begin
        [@Output(PA[i], a_0[yr,i], t, taxes=[Tax(RA,ta[i])],reference_price=1-ta_0[yr,i])]...
        [@Output(PFX, x_0[yr,i], t)]...
        [@Input(PM[m], md_0[yr,m,i], s) for m∈M]...
        @Input(PY[i], y_0[yr,i], dm)
        @Input(PFX, m_0[yr,i], dm, taxes = [Tax(RA,tm[i])],reference_price=1+tm_0[yr,i])
    end)
end

@demand(WiNnat, RA, begin
    [@final_demand(PA[i], fd_0[yr,i,:pce]) for i∈I]...
    end,begin
    [@endowment(PY[i], fs_0[yr,i]) for i∈I]...
    @endowment(PFX, bopdef_0[yr])
    [@endowment(PA[i], -sum(fd_0[yr,i,xfd] for xfd∈FD if xfd!=:pce)) for i∈I]...
    [@endowment(PVA[va], sum(va_0[yr,va,j] for j∈J)) for va∈VA]...
end)

build!(WiNnat);

# Benchmark 

In [14]:
solve!(WiNnat; :cumulative_iteration_limit => 0)

df_benchmark = generate_report(WiNnat);

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_7162.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 292

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 4.2378e-12           I 0.0e+00 1.8e-12 (income_balance[)

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 4.237844e-12
Postsolved residual: 4.2378e-12


# Counterfactual

In [15]:
fix(RA,12453.896315446877)

set_value!(ta,0)
set_value!(tm,0)

solve!(WiNnat; :cumulative_iteration_limit => 10_000)

df = generate_report(WiNnat);

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_71D0.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Zero:     3 Single:     0 Double:     0
Preprocessed size   : 288

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             5.0521e+02             0.0e+00 (zero_profit[A(:pet,))
    1     1     0   288 3.6325e+01  1.0e+00    5.1e+00 (market_clearance[PFX)
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 3.6325e+01           I 3.6e-01 3.6e+01 (market_clearanc)
    1     1     3     3 1.1756e+00  1.0e+00 SO 1.5e-01 9.1e-01 (market_clearanc)
    2     1     4     4 1.6501e-02  1.0e+00 SO 5.8e-02 1.3e-02 (market_clearanc)
    3     1     5     5 3.2813e-04  1.0e+00 SO 1.7e-03 2.8e-04 (market_clearanc)

In [16]:
df |>
    x -> sort(x, :margin, rev=true)

Row,var,value,margin
,GenericV…,Float64,Float64
1,PA[pip],0.779862,1.39383e-7
2,RA,12453.9,6.86414e-8
3,PY[oil],0.970186,2.89646e-10
4,PA[pet],0.820925,2.52783e-10
5,PVA[compen],0.9916,9.27685e-11
6,PY[pet],0.95076,5.67582e-11
7,PA[alt],0.876066,4.31442e-11
8,PA[mot],0.929959,2.94449e-11
9,PA[cep],0.957657,2.75122e-11


In [17]:
value.(Y)

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, [:ppd, :res, :com, :amb, :fbp, :rec, :con, :agr, :eec, :fnd  …  :wtt, :wpd, :wht, :wrh, :ott, :che, :air, :mmf, :otr, :min]
And data, a 71-element Vector{Float64}:
 1.0187953968971069
 1.039164510222195
 0.9992135081062237
 0.969241607811933
 1.044019876708247
 1.0255766610783168
 0.9987278473193698
 1.026509377821346
 0.9934230640379625
 1.000000000000001
 ⋮
 1.006518145195668
 1.02303098035276
 1.0194303238163016
 1.0234945511351345
 1.0054452574139956
 1.0853479140957503
 0.9969695826772511
 1.0224548485723501
 1.0168041982593308